# CIS053 Final Project - ML Titanic
Anne-Elise Chung
Hernanda Mak

## Executive Summary

todo -after everything else

## Background

The purpose of this project is to build a predictive model that can classify the sort of passenger on Titatic that will most likely survive after the shipwreck. Moreover, we aim to discover the most influential features that affect the possibility of survival among the Titanic passengers.

The sinking of the Titanic happened on April 15, 1912, where the resulting death of 1502 out of 2224 passengers led to global attention regarding safety of sea travelling and way to avoid another shipwreck of such sort. The major reason behind the disastrous outcome is the lack of lifeboats. Based on the collected data of survival history on the Titanic, we can understand and build a predictive model that would identify which type of poassenger could survive on such shipwreck.

## Survey of Existing Work

https://www.kaggle.com/code/abdmental01/exploring-data-visual-insights-unveiled#About-Function
- Author: MUHAMMAD ABDULLAH
- Techniques Used: this submission was purely showing some data visualization techniques, but it serves as a helpful comparison for the upcoming EDA work in this report.
- Results:
- ![results from data vis insights](exploring_data_vis_insights_revealed.png)

https://www.kaggle.com/code/gusthema/titanic-competition-w-tensorflow-decision-forests
- Author: GUSTHEMA
- Techniques Used: TensorFlow Decision Trees
- Results: 
- Notes: This Kaggle submission received 675 upvotes, compared to many of the other submissions that only received 2 or 3 upvotes. The popularity of this post might indicate that the results and techniques are probably sound and would be good for comparison.


https://www.kaggle.com/code/eneskosar19/titanic-sample-submission-random-forest
- Author:
- Techniques Used:
- Results:


https://www.kaggle.com/code/whitedevil4648/neural-networks-titanic
- Author: WHITEDEVIL4648
- Techniques Used: neural networks
- Results:
- Notes: 


## Our Approach


- EDA analysis
- Eliminate unuseful features and data entry
- Create classification models: ensemble trees (random forest, bagging , ADA boost), k-means clustering, decision tree classifier, SVM, logistic regression
- Standardize and Normalize to see if a more reliable model could be built
- Use cross validation to verify the accuracy of the models so to identify the best predictive model 

## Code and Results


In [1]:
import pandas as pd
from pandas import read_csv
import matplotlib.pyplot as plt
from numpy import set_printoptions
from pandas import set_option
from pandas.plotting import scatter_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split


In [3]:
test_data = read_csv('./test.csv')
train_data = read_csv('./train.csv')

In [4]:
x = train_data.drop(['PassengerId', 'Name', 'Survived'], axis=1)
y = train_data['Survived']

X_arr = x.values
Y_arr = y.values

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20)

## Conclusions
todo -after everything else

## References


## Appendix

https://github.com/aelichung/ml-titanic